# 데이터셋 준비하기
- https://www.kaggle.com/code/mitanshuchakrawarty/fine-tune-llm-for-text-summary

## 1. 환경 셋업

In [1]:
from dotenv import load_dotenv

import os

HF_TOKEN = os.getenv('HF_TOKEN')

!huggingface-cli login --token {HF_TOKEN}

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful


In [2]:
import os 
os.environ['TRANSFORMERS_CACHE'] = "/home/ec2-user/SageMaker/.cache" 
os.environ['HF_DATASETS_CACHE'] = "/home/ec2-user/SageMaker/.cache" 
os.environ['HF_HOME'] = "/home/ec2-user/SageMaker/.cache"

In [3]:
import torch
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from datasets import load_dataset, load_metric
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import warnings
warnings.filterwarnings("ignore")

/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## 2. 데이터 셋 준비
### 데이터 셋 다운로드

In [4]:
huggingface_dataset_name = "daekeun-ml/naver-news-summarization-ko"

# dataset = load_dataset(huggingface_dataset_name, "3.0.0")
dataset = load_dataset(huggingface_dataset_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [5]:
dataset["train"][0]

{'date': '2022-07-03 17:14:37',
 'category': 'economy',
 'press': 'YTN ',
 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 'link': 'https://n.news.naver.com/mne

## 3. 데이터셋 변형

### Chat Message 형태 템플릿 정의

In [6]:
import json

def format_instruction(system_prompt: str, article: str, summary: str):
    message = [
            {
                'content': system_prompt,
                'role': 'system'
            },
            {
                'content': f'Please summarize the goals for journalist in this text:\n\n{article}',
                'role': 'user'
            },
            {
                'content': f'{summary}',
                'role': 'assistant'
            }
        ]
    
    return message # json.dumps(message, indent=2) # json.dumps(message, ensure_ascii=False, indent=2)


# 사용 예시
# system_prompt = "You are an AI assistant specialized in news articles. Your role is to provide accurate summaries and insights. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings."
# article = "Within three days, the intertwined cup nest of grasses was complete, featuring a canopy of overhanging grasses to conceal it. And decades later, it served as Rinkert's portal to the past inside the California Academy of Sciences. Information gleaned from such nests, woven long ago from species in plant communities called transitional habitat, could help restore the shoreline in the future. Transitional habitat has nearly disappeared from the San Francisco Bay, and scientists need a clearer picture of its original species composition—which was never properly documented. With that insight, conservation research groups like the San Francisco Bay Bird Observatory can help guide best practices when restoring the native habitat that has long served as critical refuge for imperiled birds and animals as adjacent marshes flood more with rising sea levels. \"We can't ask restoration ecologists to plant nonnative species or to just take their best guess and throw things out there,\" says Rinkert."
# summary = "Scientists are studying nests hoping to learn about transitional habitats that could help restore the shoreline of San Francisco Bay."

# print(format_instruction(system_prompt, article, summary))

### Chat Message 형태로 변환

In [7]:
# Add system message to each conversation
columns_to_remove = list(dataset["train"].features)
columns_to_remove

['date', 'category', 'press', 'title', 'document', 'link', 'summary']

In [8]:

def generate_instruction_dataset(data_point):
    system_prompt = "You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights in Korean. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings."

    return {
        "messages": format_instruction(system_prompt, data_point["document"],data_point["summary"])
    }

def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset).remove_columns(columns_to_remove)
    )    

#### 전체 데이터 셋에서 일부 데티터 추출 (짧은 실습을 위해서)

In [9]:
train_num_debug = 10
validation_num_debug = 10
test_num_debug = 10


In [10]:
def create_message_dataset(dataset, num_train,num_val, num_test, verbose=False):
    ## APPLYING PREPROCESSING ON WHOLE DATASET

    train_dataset = process_dataset(dataset["train"].select(range(num_train)))
    validation_dataset = process_dataset(dataset["validation"].select(range(num_val)))
    test_dataset= process_dataset(dataset["test"].select(range(num_test)))
    
    if verbose:
        print(train_dataset)
        print(test_dataset)
        print(validation_dataset)

    return train_dataset,test_dataset,validation_dataset    

train_dataset,test_dataset,validation_dataset = create_message_dataset(dataset=dataset, 
                                                num_train=train_num_debug,
                                                num_val=validation_num_debug, 
                                                num_test=test_num_debug, verbose=True)    

Dataset({
    features: ['messages'],
    num_rows: 10
})
Dataset({
    features: ['messages'],
    num_rows: 10
})
Dataset({
    features: ['messages'],
    num_rows: 10
})


In [11]:
# train_dataset[0]

#### 전체 데이터 셋 저장 (성능 측정을 위해서)

In [12]:
# full_train_num = len(dataset["train"])
# full_validation_num = len(dataset["validation"])
# full_test_num = len(dataset["test"])

full_train_num = 1000
full_validation_num = 1000
full_test_num = 1000

print("train_num_samples: ", full_train_num)
print("validation_num_samples: ", full_validation_num)
print("test_num_samples: ", full_test_num)

full_train_dataset,full_test_dataset,full_validation_dataset = create_message_dataset(dataset=dataset, 
                                                                num_train=full_train_num,
                                                                num_val=full_validation_num, 
                                                                num_test=full_test_num, verbose=True)    

train_num_samples:  1000
validation_num_samples:  1000
test_num_samples:  1000
Dataset({
    features: ['messages'],
    num_rows: 1000
})
Dataset({
    features: ['messages'],
    num_rows: 1000
})
Dataset({
    features: ['messages'],
    num_rows: 1000
})


## 4. 데이터 셋을 JSON 으로 저장

In [13]:
import os

def create_dataset_json_file(huggingface_dataset_name,train_dataset, validation_dataset, test_dataset, is_full, verbose=True ):
    dataset_name = huggingface_dataset_name.split("/")[1]
    data_folder = os.path.join("../data/",dataset_name)
    os.makedirs(data_folder, exist_ok=True)

    if is_full:
        train_data_json = os.path.join(data_folder,"full_train", "train_dataset.json")
        validation_data_json = os.path.join(data_folder,"full_validation", "validation_dataset.json")
        test_data_json = os.path.join(data_folder, "full_test", "test_dataset.json")

    else:
        train_data_json = os.path.join(data_folder,"train", "train_dataset.json")
        validation_data_json = os.path.join(data_folder,"validation", "validation_dataset.json")
        test_data_json = os.path.join(data_folder, "test", "test_dataset.json")

    # save datasets to disk 
    train_dataset.to_json(train_data_json, orient="records", force_ascii=False)
    validation_dataset.to_json(validation_data_json, orient="records", force_ascii=False)
    test_dataset.to_json(test_data_json, orient="records", force_ascii=False)        

    if verbose:
        print(train_dataset)
        print(f"{train_data_json} is saved")
        print(f"{validation_data_json} is saved")
        print(f"{test_data_json} is saved")                

    return data_folder, train_data_json, validation_data_json, test_data_json



In [14]:
# Store debug dataset
data_folder, train_data_json, validation_data_json, test_data_json = create_dataset_json_file(huggingface_dataset_name=huggingface_dataset_name,
                                                                    train_dataset=train_dataset, 
                                                                    validation_dataset=validation_dataset, 
                                                                    test_dataset=test_dataset,
                                                                    is_full=False )        

# Store full dataset
data_folder, full_train_data_json, full_validation_data_json, full_test_data_json = create_dataset_json_file(huggingface_dataset_name=huggingface_dataset_name,
                                                                    train_dataset=full_train_dataset, 
                                                                    validation_dataset=full_validation_dataset, 
                                                                    test_dataset=full_test_dataset,
                                                                    is_full=True )        


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 914.99ba/s]


Dataset({
    features: ['messages'],
    num_rows: 10
})
../data/naver-news-summarization-ko/train/train_dataset.json is saved
../data/naver-news-summarization-ko/validation/validation_dataset.json is saved
../data/naver-news-summarization-ko/test/test_dataset.json is saved


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 17.43ba/s]

Dataset({
    features: ['messages'],
    num_rows: 1000
})
../data/naver-news-summarization-ko/full_train/train_dataset.json is saved
../data/naver-news-summarization-ko/full_validation/validation_dataset.json is saved
../data/naver-news-summarization-ko/full_test/test_dataset.json is saved


### 다음 노트북에서 사용하기 위해 변수 저장

In [15]:
%store data_folder
%store train_data_json 
%store validation_data_json 
%store test_data_json 
%store full_train_data_json 
%store full_validation_data_json 
%store full_test_data_json


Stored 'data_folder' (str)
Stored 'train_data_json' (str)
Stored 'validation_data_json' (str)
Stored 'test_data_json' (str)
Stored 'full_train_data_json' (str)
Stored 'full_validation_data_json' (str)
Stored 'full_test_data_json' (str)


## 5. Option: 데이터 셋을 ChatTemplate 형태로 바꾸기

### Chat Template 정의

In [16]:
LLAMA_3_CHAT_TEMPLATE = (
    "{% for message in messages %}"
        "{% if message['role'] == 'system' %}"
            "{{ message['content'] }}"
        "{% elif message['role'] == 'user' %}"
            "{{ '\n\nHuman: ' + message['content'] +  eos_token }}"
        "{% elif message['role'] == 'assistant' %}"
            "{{ '\n\nAssistant: '  + message['content'] +  eos_token  }}"
        "{% endif %}"
    "{% endfor %}"
    "{% if add_generation_prompt %}"
    "{{ '\n\nAssistant: ' }}"
    "{% endif %}"
)
LLAMA_3_CHAT_TEMPLATE

"{% for message in messages %}{% if message['role'] == 'system' %}{{ message['content'] }}{% elif message['role'] == 'user' %}{{ '\n\nHuman: ' + message['content'] +  eos_token }}{% elif message['role'] == 'assistant' %}{{ '\n\nAssistant: '  + message['content'] +  eos_token  }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '\n\nAssistant: ' }}{% endif %}"

### Chat Template 으로 변형하기

In [17]:
# Tokenizer        
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = LLAMA_3_CHAT_TEMPLATE

# template dataset
def template_dataset(examples):
    return{"text":  tokenizer.apply_chat_template(examples["messages"], tokenize=False)}

sample_train_dataset = train_dataset.map(template_dataset, remove_columns=["messages"])
# sample_test_dataset = test_dataset.map(template_dataset, remove_columns=["messages"])
# sample_validation_dataset = validation_dataset.map(template_dataset, remove_columns=["messages"])    

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 변형된 Chat Message 형태 예시 보기

In [18]:
# print random sample
import random

for index in random.sample(range(len(sample_train_dataset)), 1):
    print("index: ", index)
    # index = 5343
    print(sample_train_dataset[index]["text"])

index:  3
You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights in Korean. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings.

Human: Please summarize the goals for journalist in this text:

미국 뉴욕증시가 52년 만에 최악의 상반기를 보낸 가운데 씨티그룹이 랠리가 기대되는 종목들을 추천했다. 30일 현지시간 CNBC에 따르면 씨티은행은 팩트셋 자료를 인용해 올해 안으로 20% 이상의 주가 상승세를 보일 기업들을 꼽았다. 해당 기업들에 최소 10% 이상의 주당 순이익 성장세가 예상된다는 분석이다. 씨티그룹이 가장 먼저 꼽은 기업은 마이크로소프트다. 마이크로소프트는 올 들어 현재까지 주가가 23% 이상 하락했다. 그러나 분석가들은 주가가 현재 수준에서 36% 반등할 것으로 보고 있다. 이날 마이크로소프트는 전 거래일 대비 1.32% 하락한 256.83달러에서 거래를 마감했다. 씨티그룹은 마이크로소프트가 IT 사업 예산을 늘렸다는 점과 마이크로소프트가 추진하는 하이브리드 업무환경을 긍정적으로 평가하고 있다며 올해 주당 순이익이 16.5% 뛸 것으로 예상한다고 덧붙였다. 이외에도 씨티그룹은 엔터테인먼트와 에너지주에 주목했다. 디즈니의 주가는 올 들어 39% 급락하면서 52주 최고가 대비 반토막 났다. 씨티그룹은 디즈니의 주가가 이날 종가 94.40달러 대비 50% 반등할 수 있다며 주당 순이익도 올해 약 74%까지 오를 수 있다고 내다봤다. 씨티그룹은 에너지주의 랠리 가능성 또한 높게 점쳤다. 에너지주는 올 상반기 S P 500 지수 내 유일하게 올해 30% 이상 상승했다. 씨티그룹은 에너지주 중에서도 셰브